# Quickstart in 5 minutes

In order to run your first Deepchecks Suite all you need to have is the data and model that you wish to validate. More specifically, you need:

- DataFrames with your train and test data
- (optional) A model that supports the scikit-learn API. Required for running checks that need the model's predictions for running.

To run your first suite you need only a few lines of code, that start here [Define a Dataset Object](#Define-a-Dataset-Object).

In [1]:
# If you don't have deepchecks installed yet:
import sys
!{sys.executable} -m pip install deepchecks --quiet #--user

## Load Data, Split Train-Val, and Train a Simple Model

For the purpose of this guide we'll use the simple iris dataset and train a simple random forest model for multiclass classification:

In [2]:
# General imports
import pandas as pd
import numpy as np
np.random.seed(22)

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Load Data
iris_df = load_iris(return_X_y=False, as_frame=True)['frame']
label_col = 'target'
df_train, df_test = train_test_split(iris_df, stratify=iris_df[label_col], random_state=0)

# Train Model
rf_clf = RandomForestClassifier()
rf_clf.fit(df_train.drop(label_col, axis=1), df_train[label_col]);

## Define a Dataset Object

Initialize the Dataset object, stating the relevant metadata about the dataset (e.g. the name for the label column) <br>
Check out the Dataset's attributes to see which additional special columns can be declared and used (e.g. date column, index column).

In [3]:
from deepchecks import Dataset

# We explicitly state that this dataset has no categorical features, otherwise they will be automatically inferred
# If the dataset has categorical features, the best practice is to pass a list with their names

ds_train = Dataset(df_train, label=label_col, cat_features=[])
ds_test =  Dataset(df_test,  label=label_col, cat_features=[])

## Run the Full Suite

Use the `full_suite` that is a collection of (most of) the prebuilt checks. <br>
Check out the [list of all of the prebuilt suites](https://github.com/deepchecks/deepchecks/tree/main/deepchecks/suites) for some more info about the existing suites.

In [4]:
from deepchecks.suites import full_suite

suite = full_suite()

In [5]:
# note that we set check_datasets_policy to 'both' so that for single dataset checks (e.g. some of the integrity checks),
# both the test and the train data will be checked.

suite.run(train_dataset=ds_train, test_dataset=ds_test, model=rf_clf)

Full Suite:   0%|          | 0/36 [00:00<?, ? Check/s]

### Inspect suite and remove a condition

We can see that the single feature contribution failed, both for test and for train. Since this is a very simple dataset with few features and this behavior is not necessarily problematic, we will remove the existing conditions for the PPS

In [6]:
# Lets first print the suite to find the conditions that we want to change:
suite

Full Suite: [
	0: ModelInfo
	1: ColumnsInfo
	2: ConfusionMatrixReport
	3: PerformanceReport
		Conditions:
			0: Train-Test scores relative degradation is not greater than 0.1
	4: RocReport(excluded_classes=[])
		Conditions:
			0: AUC score for all the classes is not less than 0.7
	5: SimpleModelComparison
		Conditions:
			0: Model performance gain over simple model is not less than 10%
	6: ModelErrorAnalysis
		Conditions:
			0: The performance difference of the detected segments must not be greater than 5%
	7: CalibrationScore
	8: TrustScoreComparison
		Conditions:
			0: Mean trust score decline is not greater than 20%
	9: RegressionSystematicError
		Conditions:
			0: Bias ratio is not greater than 0.01
	10: RegressionErrorDistribution
		Conditions:
			0: Kurtosis value is not less than -0.1
	11: BoostingOverfit
		Conditions:
			0: Test score over iterations doesn't decline by more than 5% from the best score
	12: UnusedFeatures
		Conditions:
			0: Number of high variance unused featur

In [7]:
# now we can use the check's index and the condition's number to remove it:
print(suite[6])
suite[6].remove_condition(0)

ModelErrorAnalysis
	Conditions:
		0: The performance difference of the detected segments must not be greater than 5%


In [8]:
# print and see that the condition was removed
suite[6]

ModelErrorAnalysis

If we now re-run the suite, all of the existing conditions will pass.

*Note: the check we manipulated will still run as part of the Suite, however it won't appear in the Conditions Summary since it no longer has any conditions defined on it. You can still see its display results in the Additional Outputs section*

**For more info about working with conditions, see the detailed configuring conditions guide.**